In [1]:
import json
import requests
#import Flask

In [2]:
def api(endpoint):
    response = requests.get(url = endpoint)
    
    try:
        response = requests.get(endpoint, timeout=10)
        response.raise_for_status()
    # Code here will only run if the request is successful
    except requests.exceptions.HTTPError as errh:
        print(errh)
    except requests.exceptions.ConnectionError as errc:
        print(errc)
    except requests.exceptions.Timeout as errt:
        print(errt)
    except requests.exceptions.RequestException as err:
        print(err)
    data = {'api_option':'paste',
            'api_paste_code': main(),
            'api_paste_format':'python'}
    post = response.post(url = endpoint, data = data)

    return response
    

In [3]:
data_json= '''{
  "load": 480,
  "fuels":
  {
    "gas(euro/MWh)": 13.4,
    "kerosine(euro/MWh)": 50.8,
    "co2(euro/ton)": 20,
    "wind(%)": 60
  },
  "powerplants": [
    {
      "name": "gasfiredbig1",
      "type": "gasfired",
      "efficiency": 0.53,
      "pmin": 100,
      "pmax": 460
    },
    {
      "name": "gasfiredbig2",
      "type": "gasfired",
      "efficiency": 0.53,
      "pmin": 100,
      "pmax": 460
    },
    {
      "name": "gasfiredsomewhatsmaller",
      "type": "gasfired",
      "efficiency": 0.37,
      "pmin": 40,
      "pmax": 210
    },
    {
      "name": "tj1",
      "type": "turbojet",
      "efficiency": 0.3,
      "pmin": 0,
      "pmax": 16
    },
    {
      "name": "windpark1",
      "type": "windturbine",
      "efficiency": 1,
      "pmin": 0,
      "pmax": 150
    },
    {
      "name": "windpark2",
      "type": "windturbine",
      "efficiency": 1,
      "pmin": 0,
      "pmax": 36
    }
  ]
}'''

EQUATIONS
-----------
1. load = min_cost(energy_production(i))

Constraints
-----------

2a. wind: wind %  (X) pmax(i)]
2b. fuel: efficiency (X) powerplant(i) (X) fuel_cost
2c. pmin <= powerplant_production => pmax
2d. cost/pp = efficiency (X) powerplant_production (X) fuel_cost


In [4]:
data = json.loads(data_json)


In [5]:
#Extract prices
def prices():
    for i in data['fuels']:
        if 'gas' in i:
            gas_price = data['fuels'][i]
        elif 'kero' in i:
            kero_price = data['fuels'][i]
        elif 'co' in i:
            co2 = data['fuels'][i]
        elif 'wind' in i:
            wind = data['fuels'][i]
    return gas_price, kero_price, co2, wind


In [6]:
def energy_cost(gp, kp):
    #this function calculates the cost for 1MW per plant
    avg_cost = {}
    sort_avg_cost = []

    for i in data['powerplants']:
        if 'jet' in i['type']:
            avg_cost[i['name']] =  round((1 / float(i['efficiency'])) * kp,1) 

    for i in data['powerplants']:
        if 'gas' in i['type']:
            avg_cost[i['name']] =  round((1 / float(i['efficiency'])) * gp, 1) 


    sort_avg_cost = [(k,v) for k, v in avg_cost.items()]
    sort_avg_cost.sort(key= lambda x: x[1])
    return sort_avg_cost, avg_cost
#sort_avg_cost, avg_cost = energy_cost()

In [7]:
def wind_energy():
    wind_sol = []
    #set vars for calcs
    load = data['load']
    remain_load = load
    curr_load = 0 
    prod_load = 0
    wind_eff = data['fuels'].get('wind(%)') / 100  #max efficiency of wind_turb
    for i in data['powerplants']:
            #take all the free energy from wind
            if i['efficiency'] == 1:
                #check if remain_load = 0
                if remain_load != 0:
                    #check if the remain_load > wind_turb
                    if remain_load > i['pmax'] * wind_eff:
                        #calculate the value of the wind_turb
                        curr_load = round(float(i['pmax'] * wind_eff), 1)
                        remain_load -= curr_load
                        prod_load += curr_load
                        diction= {}
                        diction['name'] = i['name']
                        diction['p'] = curr_load
                        wind_sol.append(diction)
                    elif remain_load < i['pmax'] * wind_eff:
                        curr_load = remain_load
                        remain_load -= curr_load
                        diction= {}
                        diction['name'] = i['name']
                        diction['p'] = curr_load
                        wind_sol.append(diction)
    return remain_load, wind_sol
wind_list, remain_l = wind_energy()
print(wind_list, remain_l)

368.4 [{'name': 'windpark1', 'p': 90.0}, {'name': 'windpark2', 'p': 21.6}]


In [8]:
def final(remain_load, solution, sort_avg_cost):
    #use wind function: calculate remaining of free energy and create solution dict
    #remain_load, solution = wind_energy()
    #double loop to match the elements of sorted plants with the min max values
    for i in sort_avg_cost:
        #initiate dict with 0 values
        diction = {}
        diction['name'] = i[0]
        diction['p'] = 0
        solution.append(diction)

        for y in range(len(data['powerplants'])):
            if i[0] in data['powerplants'][y].values():

                #check if remain_load = 0
                if remain_load == 0:
                    break

                #check if the remaining load is < pmax of the sorted element
                elif remain_load < data['powerplants'][y]['pmax']:
                #remain_load can be produced by this plant
                    if remain_load > data['powerplants'][y]['pmin']:
                        curr_load = remain_load
                        remain_load -= curr_load
                        diction = {}
                        diction['name'] = i[0]
                        diction['p'] = curr_load
                        solution.append(diction)
                        break
                #remain_load > capacity of plant       
                elif remain_load > data['powerplants'][y]['pmax']:
                    #min capacity is satisfied
                    if remain_load > data['powerplants'][y]['pmin']: 
                        curr_load = data['powerplants'][y]['pmax']
                        diction = {}
                        diction['name'] = i[0]
                        diction['p'] = data['powerplants'][y]['pmax']
                        solution.append(diction)
                        remain_load -= curr_load
                

    return solution       


In [9]:
def main():
    gas_price, kero_price, co2, wind = prices()
    sort_avg_cost, avg_cost = energy_cost(gas_price, kero_price)
    remain_load, solution= wind_energy()
    final_json = json.dumps(final(remain_load, solution, sort_avg_cost))
    return final_json
 #   return sort_avg_cost
main()

'[{"name": "windpark1", "p": 90.0}, {"name": "windpark2", "p": 21.6}, {"name": "gasfiredbig1", "p": 0}, {"name": "gasfiredbig1", "p": 368.4}, {"name": "gasfiredbig2", "p": 0}, {"name": "gasfiredsomewhatsmaller", "p": 0}, {"name": "tj1", "p": 0}]'